In [1]:
import requests
import pyvo as vo
import astropy.units as u
from astropy.coordinates import SkyCoord
from gala.coordinates import GD1Koposov10, GD1, reflex_correct
import matplotlib.pyplot as plt
import pandas as pd

from episode_functions import *

In [2]:
name='GAIA@AIP'
url='https://gaia.aip.de/tap'
token = 'e23cb2cfd6856cf63bf330c45e31e5f0b02b943d'
tap_session = requests.Session()
tap_session.headers['Authorization'] = token
tap_service = vo.dal.TAPService(url, session=tap_session)

# Epsidoe 6: Join

* BP and RP are broad band filters - not great for CMD
* Use g and i from PanSTARRS survey (narrower band)
* Note that matching catalogs is not trivial - we're lucky that Gaia didi this for us

# Best Neighbour Table

In [27]:
ps_descrip_query = """SELECT * 
FROM tap_schema.tables 
WHERE table_name='gaiadr2.panstarrs1_best_neighbour'"

In [28]:
ps_descrip_result= tap_service.run_sync(ps_descrip_query)
ps_descrip_table = ps_descrip_result.to_table()
ps_descrip_table

gaiadr2.panstarrs1_best_neighbour
    Pan-STARRS1 best neighbour



In [30]:
ps_column_query = """SELECT column_name, description, unit 
FROM tap_schema.columns 
WHERE table_name = 'gaiadr2.panstarrs1_best_neighbour'"""
ps_column_table = tap_service.run_sync(ps_column_query)
ps_column_table.to_table()

7


In [185]:
print(len(ps_column_table))

source_id
original_ext_source_id
angular_distance
number_of_neighbours
number_of_mates
best_neighbour_multiplicity
gaia_astrometric_params


The ones we will use are:

`source_id`, which we will match up with source_id in the Gaia table.

`best_neighbour_multiplicity`, which indicates how many sources in Pan-STARRS are matched with the same probability to this source in Gaia. (should be 1)

`number_of_mates`, which indicates the number of other sources in Gaia that are matched with the same source in Pan-STARRS. (should be 0)

`original_ext_source_id`, which we will match up with obj_id in the Pan-STARRS table.

* source_id from gaia_source matched source_id from panstarrs1_best_neighbour
* original_ext_source_id in panstarrs1_best_neighbour matches obj_id in panstarrs1_original_valid
* Use join to match ra, dec, pmra, pmdec and g and i band photometry for same stars

In [32]:
ps_best_neighbor_query = """SELECT 
TOP 5
source_id, best_neighbour_multiplicity, number_of_mates, original_ext_source_id
FROM gaiadr2.panstarrs1_best_neighbour
"""

In [34]:
ps_best_neighbor_job = tap_service.run_sync(ps_best_neighbor_query)
ps_best_neighbor_job.infos

{'QUERY_STATUS': 'OK',
 'QUERY': 'SELECT \nTOP 5\nsource_id, best_neighbour_multiplicity, number_of_mates, original_ext_source_id\nFROM gaiadr2.panstarrs1_best_neighbour',
 'QUERY_LANGUAGE': 'adql-2.0',
 'SOURCE': 'gaiadr2.panstarrs1_best_neighbour'}

In [35]:
ps_best_neighbor_results = ps_best_neighbor_job.to_table()
ps_best_neighbor_results

source_id,best_neighbour_multiplicity,number_of_mates,original_ext_source_id
int64,int16,int16,int64
4151042980593547008,1,0,94052705277571961
4106041447120107392,1,0,94052825102961826
4165855566855879296,1,0,94052613725651996
4151080845023608064,1,0,94052715141551894
4106429128090826112,1,0,94052803729071947


## The Pan-STARRS table

We'll just have to talk them through this

https://datacarpentry.org/astronomy-python/06-join/index.html#the-pan-starrs-table

## Joining Tables

Bring back best practices for complex query:
* start with something you know works
* make incremental modifications
* Use TOP or COUNT

In [36]:
test_cone_query = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT('ICRS', ra, dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))
"""

In [39]:
test_cone_job = tap_service.run_sync(test_cone_query)
test_cone_job.infos

{'QUERY_STATUS': 'OK',
 'QUERY': "SELECT \nTOP 10 \nsource_id\nFROM gaiadr2.gaia_source\nWHERE 1=CONTAINS(\n  POINT('ICRS', ra, dec),\n  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))",
 'QUERY_LANGUAGE': 'adql-2.0',
 'SOURCE': 'gaiadr2.gaia_source'}

In [40]:
test_cone_results = test_cone_job.to_table()
test_cone_results

source_id
int64
3322774102494024320
3322963871329948928
3322963527734479360
3322963355933923072
3322961912826642304
3322955831151253760
3322963665173252224
3322963527732586240
3322963733891013888


start making small modifications:
* replace `source_id` with format specifier and columns array

In [41]:
cone_base_query = """SELECT 
{columns}
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT('ICRS', ra, dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))
"""

In [42]:
columns = 'source_id, ra, dec, pmra, pmdec'

cone_query = cone_base_query.format(columns=columns)
print(cone_query)

SELECT 
source_id, ra, dec, pmra, pmdec
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT('ICRS', ra, dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))



In [44]:
cone_job = tap_service.run_async(cone_query)
cone_job.infos

{'QUERY_STATUS': 'OK',
 'QUERY': "SELECT \nsource_id, ra, dec, pmra, pmdec\nFROM gaiadr2.gaia_source\nWHERE 1=CONTAINS(\n  POINT('ICRS', ra, dec),\n  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))",
 'QUERY_LANGUAGE': 'adql-2.0',
 'SOURCE': 'gaiadr2.gaia_source'}

In [45]:
cone_results = cone_job.to_table()
cone_results

source_id,ra,dec,pmra,pmdec
,Angle[deg],Angle[deg],Angular Velocity[mas/year],Angular Velocity[mas/year]
int64,float64,float64,float64,float64
3322774102494024320,88.79912516284324,7.345629133244316,-0.3846668894015961,-0.5265885869009654
3322963871329948928,88.87190097154428,7.430326542342257,1.6199268163864358,-1.1373590401698759
3322963527734479360,88.79351327982786,7.452070232913954,--,--
3322963355933923072,88.75671865195727,7.42827715750435,0.6106717155625973,-1.8107242575852163
...,...,...,...,...
3322962840537868032,88.72571439012036,7.397882741216588,2.27462723958261,-1.7203707394098302
3322963149775504896,88.7268093296086,7.417823009392743,0.6214173850631414,1.868886553066724
3322956037313221376,88.73202350548281,7.354769215883646,-2.5874360530198914,-0.333848885914519


## Adding the best neighbor table

In [60]:
neighbours_base_query = """SELECT 
{columns}
FROM gaiadr2.gaia_source AS gaia
JOIN gaiadr2.panstarrs1_best_neighbour AS best
  ON gaia.source_id = best.source_id
WHERE 1=CONTAINS(
  POINT('ICRS', gaia.ra, gaia.dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))
"""

In [65]:
neighbours_base_query = """SELECT 
{columns}
FROM gaiadr2.gaia_source AS gaia
JOIN gaiadr2.panstarrs1_best_neighbour AS best
  USING(source_id)
WHERE 1=CONTAINS(
  POINT('ICRS', gaia.ra, gaia.dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))
"""

Can use `USING` instead of `ON`

specifying table names in column names

In [66]:
column_list_neighbours = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.best_neighbour_multiplicity',
               'best.number_of_mates',
              ]
columns = ', '.join(column_list_neighbours)

neighbours_query = neighbours_base_query.format(columns=columns)
print(neighbours_query)

SELECT 
gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates
FROM gaiadr2.gaia_source AS gaia
JOIN gaiadr2.panstarrs1_best_neighbour AS best
  USING(source_id)
WHERE 1=CONTAINS(
  POINT('ICRS', gaia.ra, gaia.dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))



In [67]:
neighbours_job = tap_service.run_sync(neighbours_query)
neighbours_job.infos

{'QUERY_STATUS': 'OK',
 'QUERY': "SELECT \ngaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates\nFROM gaiadr2.gaia_source AS gaia\nJOIN gaiadr2.panstarrs1_best_neighbour AS best\n  USING(source_id)\nWHERE 1=CONTAINS(\n  POINT('ICRS', gaia.ra, gaia.dec),\n  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))",
 'QUERY_LANGUAGE': 'adql-2.0',
 'SOURCE': 'gaiadr2.panstarrs1_best_neighbour'}

In [68]:
neighbours_results = neighbours_job.to_table()
neighbours_results = rename_units(neighbours_results)
neighbours_results

source_id,ra,dec,pmra,pmdec,best_neighbour_multiplicity,number_of_mates
,deg,deg,mas / yr,mas / yr,,
int64,float64,float64,float64,float64,int16,int16
3322774102494024320,88.7991251628432,7.34562913324432,-0.384666889401596,-0.526588586900965,1,0
3322963871329948928,88.8719009715443,7.43032654234226,1.61992681638644,-1.13735904016988,1,0
3322963527734479360,88.7935132798279,7.45207023291395,--,--,1,0
3322963355933923072,88.7567186519573,7.42827715750435,0.610671715562597,-1.81072425758522,1,0
...,...,...,...,...,...,...
3322962015904151552,88.7417712864736,7.37405101687367,-0.420781805600245,-8.35323020939399,1,0
3322956037309681920,88.7368673592407,7.35875655307827,-1.86775829367713,-3.08160062661552,1,0
3322962840537868032,88.7257143901204,7.39788274121659,2.27462723958261,-1.72037073940983,1,0


note this table has fewer results because it shows only rows in both tables - some Gaia stars aren't in PanSTARRS - this is called an inner join

## Selecting by coordinate and proper motion

In [34]:
candidate_coord_pm_query_base = """SELECT 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS', {sky_point_list}))
  AND pmra BETWEEN {pmra_min} AND {pmra_max}
  AND pmdec BETWEEN {pmdec_min} AND {pmdec_max}
"""

In [36]:
columns = 'source_id, ra, dec, pmra, pmdec'

candidate_coord_pm_query = candidate_coord_pm_query_base.format(columns=columns,
                            sky_point_list=sky_point_list,
                            pmra_min=pmra_min,
                            pmra_max=pmra_max,
                            pmdec_min=pmdec_min,
                            pmdec_max=pmdec_max)

print(candidate_coord_pm_query)

SELECT 
source_id, ra, dec, pmra, pmdec
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS', 135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))
  AND pmra BETWEEN -6.7 AND -3
  AND pmdec BETWEEN -14.31 AND -11.2



In [40]:
candidate_coord_pm_job = tap_service.run_async(candidate_coord_pm_query) #note you can add the keyword queue='5m' or queue='2h' if having timeout issues
candidate_coord_pm_job.infos

{'QUERY_STATUS': 'OK',
 'QUERY': "SELECT \nsource_id, ra, dec, pmra, pmdec\nFROM gaiadr2.gaia_source\nWHERE parallax < 1\n  AND bp_rp BETWEEN -0.75 AND 2 \n  AND 1 = CONTAINS(POINT('ICRS', ra, dec), \n                   POLYGON('ICRS', 135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))\n  AND pmra BETWEEN -6.7 AND -3\n  AND pmdec BETWEEN -14.31 AND -11.2",
 'QUERY_LANGUAGE': 'adql-2.0',
 'SOURCE': 'gaiadr2.gaia_source'}

In [41]:
candidate_coord_pm_results = candidate_coord_pm_job.to_table()
candidate_coord_pm_results

source_id,ra,dec,pmra,pmdec
,Angle[deg],Angle[deg],Angular Velocity[mas/year],Angular Velocity[mas/year]
int64,float64,float64,float64,float64
684751240206357376,136.07553479476417,20.858641590222035,-6.2541465876382505,-13.553910443109421
684614969484217472,135.28016823868472,20.306664668635438,-4.028998393801843,-11.219809837537092
684662626441347328,135.1453784776957,20.330503615090926,-5.467897791172192,-12.826169997419353
684655586991056512,135.15238351660247,20.24850361286202,-3.5020415298392957,-11.585200755131742
684682525024594816,135.06212986865035,20.576247265363623,-4.9266137358470345,-11.232616892125534
684631840115621888,134.70061126129164,20.090535408044154,-4.257745508479938,-11.524211235291316
...,...,...,...,...
665022295048844288,131.89581281812244,21.810608366018453,-4.716737389193643,-11.685651072007026


### Exercise

Create a new query base called `candidate_join_query_base` that combines the `WHERE` clauses from the previous query with the `JOIN` clauses for the best neighbour. Format the query base using the column names in `column_list`, and call the result `candidate_join_query`. 
Hint: Make sure you use qualified column names everywhere!
Run your query and download the top 10 results. 

If they get an error:
* compare with neighbor

Modify to just join to panstarrs_best_neighbour - then they can try the second join if they finish but they wont be able to execute

In [69]:
candidate_join_query_base = """
SELECT 
TOP 10
{columns}
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS',gaia.ra, gaia.dec), 
                   POLYGON('ICRS',{sky_point_list}))
  AND gaia.pmra BETWEEN {pmra_min} AND {pmra_max}
  AND gaia.pmdec BETWEEN {pmdec_min} AND {pmdec_max}
"""

column_list = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.best_neighbour_multiplicity',
               'best.number_of_mates']

columns = ', '.join(column_list)

candidate_join_query = candidate_join_query_base.format(columns=columns,
                            sky_point_list= sky_point_list,
                            pmra_min=pmra_min,
                            pmra_max=pmra_max,
                            pmdec_min=pmdec_min,
                            pmdec_max=pmdec_max)
print(candidate_join_query)


candidate_join_job = tap_service.run_sync(candidate_join_query)
candidate_table = candidate_join_job.to_table()
candidate_table


SELECT 
TOP 10
gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS',gaia.ra, gaia.dec), 
                   POLYGON('ICRS',135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))
  AND gaia.pmra BETWEEN -6.7 AND -3
  AND gaia.pmdec BETWEEN -14.31 AND -11.2



source_id,ra,dec,pmra,pmdec,best_neighbour_multiplicity,number_of_mates
,Angle[deg],Angle[deg],Angular Velocity[mas/year],Angular Velocity[mas/year],,
int64,float64,float64,float64,float64,int16,int16
611742671689781632,132.600796856686,17.0438374756216,-3.60813154404558,-12.5215622443275,1,0
611914332943501440,133.350418316253,17.9010044078136,-3.18265992560615,-13.9270503672298,1,0
830805210880720256,160.101259531543,47.1911656113968,-6.34573244533553,-11.2051536083286,1,0
833919405766622464,160.355095080684,48.1516690164193,-3.42818683438447,-12.723329627089,1,0
833920883234980352,160.638894631466,48.2230281242917,-6.14357087957762,-13.9144253737475,1,0
835494490533286272,160.540184681733,48.7708134341985,-6.18475490807837,-11.6021793819245,1,0
802685308379402240,153.473512411446,39.4765363499766,-6.55887968599711,-11.8146512074032,1,0
832138373025241216,161.455661201894,47.4751418299191,-5.2819664188335,-13.2182349876583,1,0


`DALServiceError: 404 Client Error: Not Found for url: https://gaia.aip.de/tap/async/phase`  
-or-  
`DALFormatError: ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))`


indicates that too many requests are overwhelming the server

## Checking the match

best_neighbour_multiplicity: each star in Gaia how many stars in Pan-STARRS are equally likely matches

In [70]:
candidate_table['best_neighbour_multiplicity']

1
1
1
1
1
1
1
1
1
1


In [56]:
multiplicity = pd.Series(candidate_table['best_neighbour_multiplicity'])
multiplicity.describe()

count    10.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
dtype: float64

number_of_mates: number of other stars in Gaia that match with the same star in Pan-STARRS.

In [57]:
mates = pd.Series(candidate_table['number_of_mates'])
mates.describe()

count    10.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64

### Together we'll build the PanSTARRs query

In [60]:
join_solution_query_base = """
SELECT 
TOP 10
{columns}
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS',gaia.ra, gaia.dec), 
                   POLYGON('ICRS',{sky_point_list}))
  AND gaia.pmra BETWEEN {pmra_min} AND {pmra_max}
  AND gaia.pmdec BETWEEN {pmdec_min} AND {pmdec_max}
"""

column_list = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.best_neighbour_multiplicity',
               'best.number_of_mates',
               'ps.g_mean_psf_mag',
               'ps.i_mean_psf_mag']

columns = ', '.join(column_list)

join_solution_query = join_solution_query_base.format(columns=columns,
                                                      sky_point_list=sky_point_list,
                                                      pmra_min=pmra_min,
                                                      pmra_max=pmra_max,
                                                      pmdec_min=pmdec_min,
                                                      pmdec_max=pmdec_max)
print(join_solution_query)


SELECT 
TOP 10
gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates, ps.g_mean_psf_mag, ps.i_mean_psf_mag
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS',gaia.ra, gaia.dec), 
                   POLYGON('ICRS',135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))
  AND gaia.pmra BETWEEN -6.7 AND -3
  AND gaia.pmdec BETWEEN -14.31 AND -11.2



# Saving our results

In [238]:
candidate_table = rename_units(candidate_table)

In [239]:
candidate_df = make_dataframe(candidate_table)

In [241]:
filename = 'gd1_data.hdf'

candidate_df.to_hdf(filename, 'candidate_df')

In [242]:
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB

15.332862854003906

## Another file format - CSV

In [243]:
candidate_df.to_csv('gd1_data.csv')

In [244]:
getsize('gd1_data.csv') / MB

0.6899471282958984

In [246]:
read_back_csv = pd.read_csv('gd1_data.csv')

In [247]:
candidate_df.head(3)

,source_id,ra,dec,pmra,pmdec,best_neighbour_multiplicity,number_of_mates,phi1,phi2,pm_phi1,pm_phi2
0,684751240206357376,136.075535,20.858642,-6.254147,-13.553910,1,0,-58.970869,0.867534,-9.572464,1.166999
1,684614969484217472,135.280168,20.306665,-4.028998,-11.219810,1,0,-59.830924,1.213432,-6.466323,0.568633
2,684655586991056512,135.152384,20.248504,-3.502042,-11.585201,1,0,-59.943027,1.285468,-6.510768,-0.059642


In [248]:
read_back_csv.head(3)

,Unnamed: 0,source_id,ra,dec,pmra,pmdec,best_neighbour_multiplicity,number_of_mates,phi1,phi2,pm_phi1,pm_phi2
0,0,684751240206357376,136.075535,20.858642,-6.254147,-13.553910,1,0,-58.970869,0.867534,-9.572464,1.166999
1,1,684614969484217472,135.280168,20.306665,-4.028998,-11.219810,1,0,-59.830924,1.213432,-6.466323,0.568633
2,2,684655586991056512,135.152384,20.248504,-3.502042,-11.585201,1,0,-59.943027,1.285468,-6.510768,-0.059642
